# Análise Exploratória de Dados (EDA) - Detecção de Ataques de Rede

## 📡 Sobre a Coleta dos Dados

**Este dataset foi coletado manualmente em ambiente controlado para garantir a fidelidade dos cenários de rede:**

### 🔬 Metodologia de Coleta
- **Ambiente**: Rede local controlada para reprodução experimental
- **Ferramentas**: Sniffers de rede (Wireshark, tcpdump) para captura de tráfego
- **Tráfego Normal**: Capturado durante uso típico de rede (navegação, downloads, comunicação)
- **Tráfego de Ataque**: Ataques executados manualmente para simular cenários reais

### ⚙️ Cenários Reproduzidos
- **Ataques Manuais**: Executados por especialistas para garantir padrões realistas
- **Condições Controladas**: Ambiente isolado para evitar interferências
- **Diversidade de Ataques**: Múltiplos tipos de ataques para variedade no dataset
- **Reprodutibilidade**: Metodologia documentada para replicação do experimento

### 🎯 Objetivo
Criar um dataset representativo que reflita padrões reais de tráfego de rede, permitindo o desenvolvimento de modelos de detecção de intrusão eficazes em ambientes similares.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("📊 Iniciando Análise Exploratória de Dados")
print("🔬 Dataset: Tráfego de rede coletado manualmente em ambiente controlado")

In [ ]:
# 1. CARREGAMENTO E VISÃO GERAL DOS DADOS

# Carregamento dos dados processados
df = pd.read_csv('../data/processed/flows.csv')

print(f"✅ Dados carregados com sucesso!")
print(f"📈 Dimensões do dataset: {df.shape[0]} fluxos, {df.shape[1]} features")
print(f"🔍 Primeiras linhas:")
display(df.head())

In [ ]:
# 2. ESTRUTURA E TIPOS DE DADOS

print("🔬 Informações sobre os dados:")
print(f"📊 Formato: {df.shape}")
print(f"💾 Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print()

print("📋 Tipos de dados:")
display(df.dtypes.to_frame('Tipo'))
print()

print("❌ Valores ausentes:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores Ausentes': missing,
    'Percentual (%)': missing_pct
}).round(2)
display(missing_df[missing_df['Valores Ausentes'] > 0])

In [ ]:
# 3. ESTATÍSTICAS DESCRITIVAS E TESTES DE HIPÓTESE

print("📈 Resumo estatístico das features numéricas:")
numeric_cols = ['bytes', 'pkts', 'duration', 'iat_mean', 'iat_std']
desc_stats = df[numeric_cols + ['label']].describe()
display(desc_stats.round(4))

print()
print("🎯 Estatísticas por classe (Normal vs Ataque):")
stats_by_label = df.groupby('label')[numeric_cols].describe()
display(stats_by_label.round(4))

# Consumo de memória
memory_usage = df.memory_usage(deep=True).sum() / 1024**2
print(f"\n💾 Consumo de memória do DataFrame: {memory_usage:.2f} MB")

# Testes de hipótese para comparar distribuições entre classes
print("\n🔬 Testes Estatísticos (Normal vs Ataque):")
print("=" * 60)

from scipy.stats import mannwhitneyu, ks_2samp

hypothesis_results = []

for col in numeric_cols:
    normal_data = df[df['label'] == 0][col].dropna()
    attack_data = df[df['label'] == 1][col].dropna()
    
    # Teste de Mann-Whitney U (não paramétrico)
    mw_stat, mw_p = mannwhitneyu(normal_data, attack_data, alternative='two-sided')
    
    # Teste de Kolmogorov-Smirnov (distribuições)
    ks_stat, ks_p = ks_2samp(normal_data, attack_data)
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(normal_data) - 1) * normal_data.var() + 
                          (len(attack_data) - 1) * attack_data.var()) / 
                         (len(normal_data) + len(attack_data) - 2))
    cohens_d = (attack_data.mean() - normal_data.mean()) / pooled_std
    
    hypothesis_results.append({
        'Feature': col,
        'Mann_Whitney_U': mw_stat,
        'MW_p_value': mw_p,
        'MW_Significativo': 'Sim' if mw_p < 0.05 else 'Não',
        'KS_statistic': ks_stat,
        'KS_p_value': ks_p,
        'KS_Significativo': 'Sim' if ks_p < 0.05 else 'Não',
        'Cohens_d': cohens_d,
        'Effect_Size': 'Pequeno' if abs(cohens_d) < 0.5 else 'Médio' if abs(cohens_d) < 0.8 else 'Grande'
    })

hypothesis_df = pd.DataFrame(hypothesis_results)
display(hypothesis_df.round(4))

print("\n🔍 Interpretação dos Testes:")
print("• Mann-Whitney U: Testa se as medianas são diferentes (não paramétrico)")
print("• Kolmogorov-Smirnov: Testa se as distribuições são diferentes")
print("• Cohen's d: Mede o tamanho do efeito (diferença padronizada)")
print("  - |d| < 0.5: Efeito pequeno")
print("  - 0.5 ≤ |d| < 0.8: Efeito médio") 
print("  - |d| ≥ 0.8: Efeito grande")

# Resumo das features mais discriminativas
significant_features = hypothesis_df[
    (hypothesis_df['MW_Significativo'] == 'Sim') & 
    (hypothesis_df['KS_Significativo'] == 'Sim')
]['Feature'].tolist()

print(f"\n⭐ Features estatisticamente significativas: {len(significant_features)}/{len(numeric_cols)}")
for feature in significant_features:
    effect = hypothesis_df[hypothesis_df['Feature'] == feature]['Effect_Size'].iloc[0]
    cohens = hypothesis_df[hypothesis_df['Feature'] == feature]['Cohens_d'].iloc[0]
    print(f"  • {feature}: {effect} efeito (d = {cohens:.3f})")

if len(significant_features) < len(numeric_cols):
    non_significant = [f for f in numeric_cols if f not in significant_features]
    print(f"\n⚠️ Features sem diferença significativa: {non_significant}")
    print("   Considere removê-las ou combiná-las para melhorar o modelo")

In [ ]:
# 4. ANÁLISE DE BALANCEAMENTO DAS CLASSES

print("⚖️ Distribuição das classes:")
label_counts = df['label'].value_counts().sort_index()
label_props = df['label'].value_counts(normalize=True).sort_index()

balance_df = pd.DataFrame({
    'Classe': ['Normal (0)', 'Ataque (1)'],
    'Quantidade': label_counts.values,
    'Proporção (%)': (label_props.values * 100).round(2)
})

display(balance_df)

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico de barras
balance_df.plot(x='Classe', y='Quantidade', kind='bar', ax=ax1, color=['skyblue', 'salmon'])
ax1.set_title('📊 Distribuição Absoluta das Classes')
ax1.set_xlabel('Classe')
ax1.set_ylabel('Número de Fluxos')
ax1.tick_params(axis='x', rotation=0)

# Gráfico de pizza
ax2.pie(balance_df['Quantidade'], labels=balance_df['Classe'], autopct='%1.1f%%', 
        colors=['skyblue', 'salmon'], startangle=90)
ax2.set_title('🥧 Proporção das Classes')

plt.tight_layout()
plt.show()

# Análise de balanceamento
ratio = label_counts.iloc[1] / label_counts.iloc[0]
print(f"📊 Proporção Ataque/Normal: {ratio:.3f}")
if ratio < 0.1:
    print("⚠️  Dataset muito desbalanceado - considere técnicas de balanceamento")
elif ratio < 0.5:
    print("⚠️  Dataset moderadamente desbalanceado")
else:
    print("✅ Dataset relativamente balanceado")

In [ ]:
# 5. DISTRIBUIÇÕES UNIVARIADAS E ANÁLISE DE TRANSFORMAÇÕES

print("📊 Analisando distribuições das features numéricas...")

from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer

fig, axes = plt.subplots(4, 2, figsize=(15, 16))
axes = axes.ravel()

transformation_results = []

for i, col in enumerate(numeric_cols):
    # Plot original
    axes[i*2].hist(df[col], bins=50, alpha=0.7, density=True, color='skyblue', edgecolor='black')
    
    # Estatísticas da distribuição original
    mean_val = df[col].mean()
    median_val = df[col].median()
    skew_val = df[col].skew()
    
    # Linhas de referência
    axes[i*2].axvline(mean_val, color='red', linestyle='--', label=f'Média: {mean_val:.2f}')
    axes[i*2].axvline(median_val, color='green', linestyle='--', label=f'Mediana: {median_val:.2f}')
    
    axes[i*2].set_title(f'📈 {col} (Original)\nSkew: {skew_val:.2f}')
    axes[i*2].set_xlabel(col)
    axes[i*2].set_ylabel('Densidade')
    axes[i*2].legend()
    axes[i*2].grid(True, alpha=0.3)
    
    # Teste de normalidade original
    _, p_value_orig = stats.normaltest(df[col].dropna())
    normality_orig = "Normal" if p_value_orig > 0.05 else "Não Normal"
    
    # Aplicar transformação se skew > 1
    best_transformation = "Nenhuma"
    transformed_data = df[col].copy()
    skew_transformed = skew_val
    normality_transformed = normality_orig
    
    if abs(skew_val) > 1:
        # Testar diferentes transformações
        transformations = {}
        
        # Log transform (apenas para valores positivos)
        if (df[col] > 0).all():
            log_data = np.log1p(df[col])  # log(1+x) para evitar log(0)
            transformations['Log(1+x)'] = log_data
        
        # Square root transform (apenas para valores não negativos)
        if (df[col] >= 0).all():
            sqrt_data = np.sqrt(df[col])
            transformations['Sqrt'] = sqrt_data
        
        # Box-Cox transform (apenas para valores positivos)
        if (df[col] > 0).all():
            try:
                boxcox_data, lambda_param = boxcox(df[col])
                transformations[f'Box-Cox(λ={lambda_param:.3f})'] = boxcox_data
            except:
                pass
        
        # Yeo-Johnson transform (aceita valores negativos)
        try:
            pt = PowerTransformer(method='yeo-johnson', standardize=False)
            yj_data = pt.fit_transform(df[col].values.reshape(-1, 1)).flatten()
            transformations['Yeo-Johnson'] = yj_data
        except:
            pass
        
        # Escolher a melhor transformação (menor skew absoluto)
        if transformations:
            best_skew = float('inf')
            for transform_name, transform_data in transformations.items():
                current_skew = abs(stats.skew(transform_data))
                if current_skew < best_skew:
                    best_skew = current_skew
                    best_transformation = transform_name
                    transformed_data = transform_data
                    skew_transformed = stats.skew(transform_data)
    
    # Plot transformado
    axes[i*2+1].hist(transformed_data, bins=50, alpha=0.7, density=True, 
                     color='lightcoral', edgecolor='black')
    
    mean_trans = transformed_data.mean()
    median_trans = transformed_data.median()
    
    axes[i*2+1].axvline(mean_trans, color='red', linestyle='--', label=f'Média: {mean_trans:.2f}')
    axes[i*2+1].axvline(median_trans, color='green', linestyle='--', label=f'Mediana: {median_trans:.2f}')
    
    # Teste de normalidade transformado
    if best_transformation != "Nenhuma":
        _, p_value_trans = stats.normaltest(transformed_data.dropna())
        normality_transformed = "Normal" if p_value_trans > 0.05 else "Não Normal"
    
    axes[i*2+1].set_title(f'🔄 {col} ({best_transformation})\nSkew: {skew_transformed:.2f}')
    axes[i*2+1].set_xlabel(f'{col} (transformado)')
    axes[i*2+1].set_ylabel('Densidade')
    axes[i*2+1].legend()
    axes[i*2+1].grid(True, alpha=0.3)
    
    # Armazenar resultados
    transformation_results.append({
        'Feature': col,
        'Skew_Original': skew_val,
        'Normalidade_Original': normality_orig,
        'Transformação_Recomendada': best_transformation,
        'Skew_Transformado': skew_transformed,
        'Normalidade_Transformada': normality_transformed,
        'Melhoria_Skew': abs(skew_val) - abs(skew_transformed)
    })

plt.tight_layout()
plt.show()

# Resumo das transformações
transformation_df = pd.DataFrame(transformation_results)
print("\n🔄 Resumo das Transformações Recomendadas:")
display(transformation_df.round(3))

print("\n💡 Recomendações para Pré-processamento:")
high_skew_features = transformation_df[abs(transformation_df['Skew_Original']) > 1]
if len(high_skew_features) > 0:
    print("⚠️ Features com alta assimetria (|skew| > 1):")
    for _, row in high_skew_features.iterrows():
        improvement = "✅ Melhorou" if row['Melhoria_Skew'] > 0.5 else "⚠️ Pouca melhoria"
        print(f"  • {row['Feature']}: {row['Transformação_Recomendada']} - {improvement}")
        print(f"    Skew: {row['Skew_Original']:.2f} → {row['Skew_Transformado']:.2f}")
else:
    print("✅ Todas as features têm assimetria aceitável")

print("\n🎯 Estratégia para Autoencoder:")
print("• Features com skew > 2: Aplicar transformação antes da normalização")
print("• Features normalizadas: StandardScaler ou MinMaxScaler")
print("• Manter transformações inversíveis para interpretação dos resultados")

In [ ]:
# 6. ANÁLISE COMPARATIVA POR CLASSE (NORMAL VS ATAQUE)

print("🎯 Comparando distribuições entre classes Normal e Ataque...")

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    # Boxplot comparativo
    box_data = [df[df['label'] == 0][col].dropna(), df[df['label'] == 1][col].dropna()]
    box = axes[i].boxplot(box_data, labels=['Normal', 'Ataque'], patch_artist=True)
    
    # Colorir as caixas
    colors = ['lightblue', 'lightcoral']
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
    
    axes[i].set_title(f'📦 {col} por Classe')
    axes[i].set_ylabel(col)
    axes[i].grid(True, alpha=0.3)
    
    # Estatísticas comparativas
    normal_mean = df[df['label'] == 0][col].mean()
    attack_mean = df[df['label'] == 1][col].mean()
    
    # Teste t para diferença de médias
    _, p_value = stats.ttest_ind(
        df[df['label'] == 0][col].dropna(), 
        df[df['label'] == 1][col].dropna()
    )
    
    significance = "Significativa" if p_value < 0.05 else "Não Significativa"
    
    axes[i].text(0.02, 0.98, 
                f'Normal: {normal_mean:.2f}\nAtaque: {attack_mean:.2f}\nDif: {significance}', 
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

# Remove o subplot extra
if len(numeric_cols) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

# Análise quantitativa das diferenças
print("\n📊 Resumo das diferenças entre classes:")
comparison_stats = []

for col in numeric_cols:
    normal_data = df[df['label'] == 0][col]
    attack_data = df[df['label'] == 1][col]
    
    _, p_value = stats.ttest_ind(normal_data.dropna(), attack_data.dropna())
    effect_size = (attack_data.mean() - normal_data.mean()) / df[col].std()
    
    comparison_stats.append({
        'Feature': col,
        'Normal_Mean': normal_data.mean(),
        'Attack_Mean': attack_data.mean(),
        'Diferença_%': ((attack_data.mean() - normal_data.mean()) / normal_data.mean() * 100),
        'P_Value': p_value,
        'Effect_Size': effect_size
    })

comparison_df = pd.DataFrame(comparison_stats)
display(comparison_df.round(4))

print("\n🔍 Features mais discriminativas (por tamanho do efeito):")
top_features = comparison_df.reindex(comparison_df['Effect_Size'].abs().sort_values(ascending=False).index)
for _, row in top_features.head(3).iterrows():
    print(f"• {row['Feature']}: Effect Size = {row['Effect_Size']:.3f}")

In [ ]:
# 7. ANÁLISE DE CORRELAÇÕES E MULTICOLINEARIDADE

print("🔗 Analisando correlações entre features...")

# Matriz de correlação
corr_matrix = df[numeric_cols].corr()

# Visualização da matriz de correlação
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Heatmap principal
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, ax=ax1)
ax1.set_title('🌡️ Matriz de Correlação (Pearson)')

# Heatmap apenas das correlações fortes (|r| > 0.5)
strong_corr = corr_matrix.copy()
strong_corr[abs(strong_corr) < 0.5] = 0
sns.heatmap(strong_corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, ax=ax2)
ax2.set_title('🔥 Correlações Fortes (|r| ≥ 0.5)')

plt.tight_layout()
plt.show()

# Identificar correlações mais fortes
print("\n📈 Correlações mais fortes entre features:")
# Criar matriz triangular superior para evitar duplicatas
upper_triangle = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
correlation_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if upper_triangle[i, j]:
            feature1 = corr_matrix.columns[i]
            feature2 = corr_matrix.columns[j]
            correlation = corr_matrix.iloc[i, j]
            correlation_pairs.append({
                'Feature_1': feature1,
                'Feature_2': feature2,
                'Correlação': correlation,
                'Magnitude': abs(correlation)
            })

correlation_df = pd.DataFrame(correlation_pairs)
correlation_df = correlation_df.sort_values('Magnitude', ascending=False)

print("Top 5 correlações mais fortes:")
display(correlation_df.head().round(3))

# Análise crítica de multicolinearidade
print("\n⚠️ Análise Crítica de Multicolinearidade:")
print("=" * 60)

# VIF (Variance Inflation Factor)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Preparar dados para VIF (remover valores infinitos/nulos)
X_vif = df[numeric_cols].copy()
X_vif = X_vif.replace([np.inf, -np.inf], np.nan).dropna()

if len(X_vif) > 0:
    # Calcular VIF para cada feature
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_vif.columns
    vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) 
                       for i in range(len(X_vif.columns))]
    vif_data = vif_data.sort_values('VIF', ascending=False)
    
    print("📊 Variance Inflation Factor (VIF):")
    display(vif_data.round(2))
    
    print("\n🔍 Interpretação do VIF:")
    print("• VIF < 5: Multicolinearidade baixa")
    print("• 5 ≤ VIF < 10: Multicolinearidade moderada")
    print("• VIF ≥ 10: Multicolinearidade alta (problema)")
    
    # Identificar features problemáticas
    high_vif = vif_data[vif_data['VIF'] >= 10]
    moderate_vif = vif_data[(vif_data['VIF'] >= 5) & (vif_data['VIF'] < 10)]
    
    if len(high_vif) > 0:
        print(f"\n🚨 Features com multicolinearidade alta (VIF ≥ 10):")
        for _, row in high_vif.iterrows():
            print(f"  • {row['Feature']}: VIF = {row['VIF']:.2f}")
        print("  → Recomendação: Remover uma das features correlacionadas")
    
    if len(moderate_vif) > 0:
        print(f"\n⚠️ Features com multicolinearidade moderada (5 ≤ VIF < 10):")
        for _, row in moderate_vif.iterrows():
            print(f"  • {row['Feature']}: VIF = {row['VIF']:.2f}")
        print("  → Recomendação: Monitorar durante a modelagem")
    
    if len(high_vif) == 0 and len(moderate_vif) == 0:
        print("\n✅ Todas as features têm VIF < 5 - Multicolinearidade aceitável")

else:
    print("❌ Não foi possível calcular VIF devido a dados inconsistentes")

# Análise de correlações extremas (|r| > 0.9)
print(f"\n🔥 Correlações Extremas (|r| > 0.9):")
extreme_corr = correlation_df[correlation_df['Magnitude'] > 0.9]

if len(extreme_corr) > 0:
    print("Features com correlação quase perfeita:")
    for _, row in extreme_corr.iterrows():
        direction = "positiva" if row['Correlação'] > 0 else "negativa"
        print(f"• {row['Feature_1']} ↔ {row['Feature_2']}: {row['Correlação']:.3f} ({direction})")
    
    print("\n💡 Recomendações para Modelagem:")
    print("• Considere remover uma das features em cada par altamente correlacionado")
    print("• Ou combine-as usando PCA ou média ponderada")
    print("• Para autoencoders, alta correlação pode causar redundância no aprendizado")
    
else:
    print("✅ Nenhuma correlação extrema detectada")

# Análise de correlações fortes (0.7 ≤ |r| < 0.9)
strong_corr_pairs = correlation_df[(correlation_df['Magnitude'] >= 0.7) & 
                                   (correlation_df['Magnitude'] < 0.9)]

if len(strong_corr_pairs) > 0:
    print(f"\n🔶 Correlações Fortes (0.7 ≤ |r| < 0.9):")
    for _, row in strong_corr_pairs.iterrows():
        direction = "positiva" if row['Correlação'] > 0 else "negativa"
        print(f"• {row['Feature_1']} ↔ {row['Feature_2']}: {row['Correlação']:.3f} ({direction})")

# Matriz de correlação filtrada para visualização crítica
if len(extreme_corr) > 0 or len(strong_corr_pairs) > 0:
    print(f"\n📊 Visualização das Correlações Críticas:")
    
    # Criar máscara para correlações críticas
    critical_corr = corr_matrix.copy()
    critical_corr[abs(critical_corr) < 0.7] = 0
    
    plt.figure(figsize=(10, 8))
    mask = critical_corr == 0
    sns.heatmap(critical_corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
                square=True, linewidths=0.5, mask=mask, 
                cbar_kws={"shrink": .8})
    plt.title('🎯 Correlações Críticas (|r| ≥ 0.7)')
    plt.tight_layout()
    plt.show()

# Interpretação das correlações para o contexto de rede
print("\n🌐 Interpretação para Detecção de Ataques:")
for _, row in correlation_df.head(3).iterrows():
    corr_val = row['Correlação']
    if abs(corr_val) >= 0.7:
        strength = "muito forte"
    elif abs(corr_val) >= 0.5:
        strength = "forte"
    elif abs(corr_val) >= 0.3:
        strength = "moderada"
    else:
        strength = "fraca"
    
    direction = "positiva" if corr_val > 0 else "negativa"
    print(f"• {row['Feature_1']} ↔ {row['Feature_2']}: Correlação {strength} {direction} ({corr_val:.3f})")
    
    # Interpretação contextual
    if 'bytes' in [row['Feature_1'], row['Feature_2']] and 'pkts' in [row['Feature_1'], row['Feature_2']]:
        print("  → Fluxos com mais pacotes tendem a transferir mais bytes (esperado)")
    elif 'duration' in [row['Feature_1'], row['Feature_2']]:
        print("  → Duração do fluxo relacionada com volume de dados (esperado)")
    elif 'iat' in row['Feature_1'].lower() or 'iat' in row['Feature_2'].lower():
        print("  → Padrão temporal pode indicar comportamento automatizado vs humano")

print(f"\n🎯 Recomendações Finais para Seleção de Features:")
print("• Priorizar features com baixa correlação entre si (< 0.8)")
print("• Manter features com alta discriminação entre classes")
print("• Considerar engenharia de features para reduzir multicolinearidade")
print("• Validar seleção final com validação cruzada")

In [ ]:
# 8. ANÁLISE BIVARIADA E REDUÇÃO DE DIMENSIONALIDADE

print("🎨 Criando scatter plots para visualizar relações entre features...")

# Pairplot com distinção por classe
g = sns.pairplot(data=df[numeric_cols + ['label']], 
                 hue='label', 
                 plot_kws={'alpha': 0.6, 's': 30},
                 diag_kind='hist',
                 palette=['skyblue', 'salmon'])

# Personalizar o plot
g.fig.suptitle('🔍 Análise Bivariada - Relações entre Features por Classe', 
               y=1.02, fontsize=16, fontweight='bold')

# Adicionar legendas personalizadas
for ax in g.axes.flat:
    if ax.legend_:
        ax.legend(labels=['Normal', 'Ataque'], loc='best')

plt.show()

# Análise PCA para visualização de clusters
print("\n🔍 Análise de Componentes Principais (PCA)")
print("=" * 50)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Preparar dados para PCA
X = df[numeric_cols].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Variância explicada
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"📊 Variância explicada por componente:")
for i, var in enumerate(explained_variance):
    print(f"  PC{i+1}: {var:.3f} ({var*100:.1f}%)")

print(f"\n📈 Variância acumulada:")
for i, cum_var in enumerate(cumulative_variance):
    print(f"  PC1-PC{i+1}: {cum_var:.3f} ({cum_var*100:.1f}%)")

# Visualizações PCA
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Variância explicada
ax1.bar(range(1, len(explained_variance)+1), explained_variance, alpha=0.7, color='skyblue')
ax1.plot(range(1, len(cumulative_variance)+1), cumulative_variance, 'ro-', color='red')
ax1.set_xlabel('Componente Principal')
ax1.set_ylabel('Variância Explicada')
ax1.set_title('📊 Variância Explicada por Componente')
ax1.grid(True, alpha=0.3)
ax1.legend(['Acumulada', 'Individual'])

# 2. Scatter plot PC1 vs PC2
normal_mask = df['label'] == 0
attack_mask = df['label'] == 1

ax2.scatter(X_pca[normal_mask, 0], X_pca[normal_mask, 1], 
           alpha=0.6, c='skyblue', label='Normal', s=30)
ax2.scatter(X_pca[attack_mask, 0], X_pca[attack_mask, 1], 
           alpha=0.6, c='salmon', label='Ataque', s=30)
ax2.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
ax2.set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
ax2.set_title('🎯 Projeção PCA (PC1 vs PC2)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Loading plot (contribuição das features)
loadings = pca.components_[:2].T * np.sqrt(pca.explained_variance_[:2])
ax3.scatter(loadings[:, 0], loadings[:, 1], alpha=0.7, s=100, color='purple')
for i, feature in enumerate(numeric_cols):
    ax3.annotate(feature, (loadings[i, 0], loadings[i, 1]), 
                xytext=(5, 5), textcoords='offset points', fontsize=10)
ax3.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
ax3.set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
ax3.set_title('🎯 Loading Plot (Contribuição das Features)')
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax3.axvline(x=0, color='k', linestyle='--', alpha=0.5)

# 4. PC1 vs PC3
ax4.scatter(X_pca[normal_mask, 0], X_pca[normal_mask, 2], 
           alpha=0.6, c='skyblue', label='Normal', s=30)
ax4.scatter(X_pca[attack_mask, 0], X_pca[attack_mask, 2], 
           alpha=0.6, c='salmon', label='Ataque', s=30)
ax4.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
ax4.set_ylabel(f'PC3 ({explained_variance[2]*100:.1f}%)')
ax4.set_title('🎯 Projeção PCA (PC1 vs PC3)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análise das componentes principais
print("\n🔍 Interpretação das Componentes Principais:")
feature_contributions = pd.DataFrame(
    pca.components_[:3].T, 
    columns=[f'PC{i+1}' for i in range(3)], 
    index=numeric_cols
)
display(feature_contributions.round(3))

print("\n💡 Insights do PCA:")
print(f"• PC1 e PC2 explicam {(cumulative_variance[1]*100):.1f}% da variância")
print(f"• Para capturar 95% da variância, precisamos de {np.argmax(cumulative_variance >= 0.95) + 1} componentes")

# Avaliar separabilidade das classes no espaço PCA
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X_pca[:, :2], df['label'])
print(f"• Silhouette Score (PC1-PC2): {silhouette_avg:.3f}")

if silhouette_avg > 0.5:
    print("  ✅ Boa separabilidade entre classes no espaço PCA")
elif silhouette_avg > 0.3:
    print("  ⚠️ Separabilidade moderada entre classes")
else:
    print("  ❌ Baixa separabilidade - classes sobrepostas")

# Scatter plots individuais para as top 3 correlações
if len(correlation_df) >= 3:
    print("\n🔍 Análise Detalhada das Principais Correlações:")
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for i in range(3):
        row = correlation_df.iloc[i]
        feature1, feature2 = row['Feature_1'], row['Feature_2']
        
        # Scatter plot colorido por classe
        normal_data = df[df['label'] == 0]
        attack_data = df[df['label'] == 1]
        
        axes[i].scatter(normal_data[feature1], normal_data[feature2], 
                       alpha=0.6, c='skyblue', label='Normal', s=30)
        axes[i].scatter(attack_data[feature1], attack_data[feature2], 
                       alpha=0.6, c='salmon', label='Ataque', s=30)
        
        axes[i].set_xlabel(feature1)
        axes[i].set_ylabel(feature2)
        axes[i].set_title(f'{feature1} vs {feature2}\n(r = {row["Correlação"]:.3f})')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("\n🎯 Recomendações para Modelagem:")
print("• Use PCA se o autoencoder tiver dificuldades com a dimensionalidade")
print("• As 2-3 primeiras componentes capturam a maior parte da informação")
print("• Considere usar features transformadas para melhor separabilidade")

In [ ]:
# 9. DETECÇÃO AVANÇADA DE OUTLIERS

print("🔍 Detectando e analisando outliers com múltiplos métodos...")

def detect_outliers_iqr(data, column):
    """Detecta outliers usando o método IQR (Interquartile Range)"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

def detect_outliers_zscore(data, column, threshold=3):
    """Detecta outliers usando Z-score"""
    z_scores = np.abs(stats.zscore(data[column].dropna()))
    outlier_indices = data[column].dropna().index[z_scores > threshold]
    outliers = data.loc[outlier_indices]
    return outliers, threshold

def detect_outliers_modified_zscore(data, column, threshold=3.5):
    """Detecta outliers usando Modified Z-score (mais robusto)"""
    median = data[column].median()
    mad = np.median(np.abs(data[column] - median))
    modified_z_scores = 0.6745 * (data[column] - median) / mad
    outlier_indices = data[np.abs(modified_z_scores) > threshold].index
    outliers = data.loc[outlier_indices]
    return outliers, threshold

# Análise comparativa de métodos de detecção
outlier_comparison = []
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    # Método 1: IQR
    iqr_outliers, iqr_lower, iqr_upper = detect_outliers_iqr(df, col)
    
    # Método 2: Z-score
    zscore_outliers, zscore_thresh = detect_outliers_zscore(df, col, threshold=3)
    
    # Método 3: Modified Z-score
    mod_zscore_outliers, mod_thresh = detect_outliers_modified_zscore(df, col, threshold=3.5)
    
    # Interseção dos métodos (outliers mais robustos)
    iqr_indices = set(iqr_outliers.index)
    zscore_indices = set(zscore_outliers.index)
    mod_zscore_indices = set(mod_zscore_outliers.index)
    
    # Outliers detectados por pelo menos 2 métodos
    consensus_outliers = iqr_indices.intersection(zscore_indices).union(
        iqr_indices.intersection(mod_zscore_indices)).union(
        zscore_indices.intersection(mod_zscore_indices))
    
    outlier_comparison.append({
        'Feature': col,
        'IQR_Outliers': len(iqr_outliers),
        'IQR_Percent': (len(iqr_outliers) / len(df)) * 100,
        'ZScore_Outliers': len(zscore_outliers),
        'ZScore_Percent': (len(zscore_outliers) / len(df)) * 100,
        'ModZScore_Outliers': len(mod_zscore_outliers),
        'ModZScore_Percent': (len(mod_zscore_outliers) / len(df)) * 100,
        'Consensus_Outliers': len(consensus_outliers),
        'Consensus_Percent': (len(consensus_outliers) / len(df)) * 100,
        'Normal_Consensus': len([idx for idx in consensus_outliers if df.loc[idx, 'label'] == 0]),
        'Attack_Consensus': len([idx for idx in consensus_outliers if df.loc[idx, 'label'] == 1])
    })
    
    # Visualização
    axes[i].boxplot([df[df['label'] == 0][col], df[df['label'] == 1][col]], 
                   labels=['Normal', 'Ataque'], patch_artist=True)
    
    # Destacar outliers consensus
    if consensus_outliers:
        consensus_data = df.loc[list(consensus_outliers)]
        normal_consensus = consensus_data[consensus_data['label'] == 0][col]
        attack_consensus = consensus_data[consensus_data['label'] == 1][col]
        
        if len(normal_consensus) > 0:
            axes[i].scatter([1] * len(normal_consensus), normal_consensus, 
                           color='darkblue', alpha=0.8, s=50, marker='x', 
                           label=f'Outliers Normal ({len(normal_consensus)})')
        if len(attack_consensus) > 0:
            axes[i].scatter([2] * len(attack_consensus), attack_consensus, 
                           color='darkred', alpha=0.8, s=50, marker='x',
                           label=f'Outliers Ataque ({len(attack_consensus)})')
    
    axes[i].set_title(f'📦 {col}\nConsensus Outliers: {len(consensus_outliers)} ({(len(consensus_outliers)/len(df)*100):.1f}%)')
    axes[i].grid(True, alpha=0.3)
    if consensus_outliers:
        axes[i].legend()

plt.tight_layout()
plt.show()

# Resumo comparativo dos métodos
outlier_df = pd.DataFrame(outlier_comparison)
print("\n📊 Comparação de Métodos de Detecção de Outliers:")
display(outlier_df.round(2))

# Análise estatística dos outliers consensus
print("\n🔍 Análise Detalhada dos Outliers Consensus:")
print("=" * 60)

total_consensus = outlier_df['Consensus_Outliers'].sum()
total_normal_consensus = outlier_df['Normal_Consensus'].sum()
total_attack_consensus = outlier_df['Attack_Consensus'].sum()

print(f"• Total de outliers consensus: {total_consensus}")
print(f"• Outliers em tráfego normal: {total_normal_consensus} ({(total_normal_consensus/total_consensus*100):.1f}%)")
print(f"• Outliers em tráfego de ataque: {total_attack_consensus} ({(total_attack_consensus/total_consensus*100):.1f}%)")

# Identificar fluxos que são outliers em múltiplas features
print("\n🎯 Fluxos Outliers em Múltiplas Features:")
outlier_counts_per_flow = {}

for col in numeric_cols:
    iqr_outliers, _, _ = detect_outliers_iqr(df, col)
    zscore_outliers, _ = detect_outliers_zscore(df, col, threshold=3)
    
    consensus_indices = set(iqr_outliers.index).intersection(set(zscore_outliers.index))
    
    for idx in consensus_indices:
        outlier_counts_per_flow[idx] = outlier_counts_per_flow.get(idx, 0) + 1

# Fluxos que são outliers em múltiplas features
multi_feature_outliers = {k: v for k, v in outlier_counts_per_flow.items() if v >= 2}

if multi_feature_outliers:
    print(f"• Fluxos outliers em 2+ features: {len(multi_feature_outliers)}")
    
    # Analisar características desses fluxos
    extreme_outlier_indices = list(multi_feature_outliers.keys())
    extreme_outliers_df = df.loc[extreme_outlier_indices]
    
    print("\n📈 Características dos Outliers Extremos:")
    extreme_stats = extreme_outliers_df.groupby('label')[numeric_cols].agg(['count', 'mean', 'median']).round(2)
    display(extreme_stats)
    
    # Top 5 fluxos mais anômalos
    sorted_outliers = sorted(multi_feature_outliers.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"\n🔥 Top 5 Fluxos Mais Anômalos:")
    for idx, count in sorted_outliers:
        label_name = "Ataque" if df.loc[idx, 'label'] == 1 else "Normal"
        print(f"  • Fluxo {idx}: Outlier em {count} features ({label_name})")
else:
    print("• Nenhum fluxo é outlier em múltiplas features")

# Análise Z-score extremo (|z| > 4)
print("\n⚠️ Outliers Extremos (|Z-score| > 4):")
extreme_found = False

for col in numeric_cols:
    z_scores = np.abs(stats.zscore(df[col].dropna()))
    extreme_mask = z_scores > 4
    
    if extreme_mask.any():
        extreme_found = True
        extreme_indices = df[col].dropna().index[extreme_mask]
        normal_extreme = sum(df.loc[extreme_indices, 'label'] == 0)
        attack_extreme = sum(df.loc[extreme_indices, 'label'] == 1)
        max_zscore = z_scores.max()
        
        print(f"• {col}: {extreme_mask.sum()} outliers extremos (|Z| max: {max_zscore:.2f})")
        print(f"  Normal: {normal_extreme}, Ataque: {attack_extreme}")

if not extreme_found:
    print("✅ Nenhum outlier extremo detectado (|Z-score| > 4)")

print("\n💡 Recomendações para Pré-processamento:")
high_outlier_features = outlier_df[outlier_df['Consensus_Percent'] > 5]

if len(high_outlier_features) > 0:
    print("⚠️ Features com muitos outliers consensus (>5%):")
    for _, row in high_outlier_features.iterrows():
        print(f"   • {row['Feature']}: {row['Consensus_Percent']:.1f}%")
        if row['Attack_Consensus'] > row['Normal_Consensus']:
            print(f"     → Outliers majoritariamente em ataques - MANTER para detecção")
        else:
            print(f"     → Outliers balanceados - considerar transformação ou remoção")
else:
    print("✅ Percentual de outliers aceitável em todas as features")

print("\n🎯 Estratégia para Autoencoder:")
print("• Treinar apenas com dados normais (sem outliers)")
print("• Usar outliers consensus para validação da detecção de anomalias")
print("• Manter outliers de ataque para teste do modelo")
print("• Considerar remoção apenas de outliers extremos do tráfego normal")

# 10. CONCLUSÕES E ROADMAP PARA MODELAGEM

## 🎯 Principais Descobertas da EDA

### Estrutura dos Dados
- **Tamanho do dataset**: Análise completa de fluxos e features
- **Qualidade dos dados**: Verificação de valores ausentes e integridade
- **Balanceamento**: Distribuição quantificada entre tráfego normal e de ataque

### Features Mais Discriminativas (Baseado em Testes Estatísticos)
- **Significância Estatística**: Features com diferenças significativas entre classes
- **Effect Size**: Medição do impacto discriminativo de cada feature
- **Power Analysis**: Identificação das features mais importantes para o modelo

### Padrões Identificados
- **Distribuições**: Análise de normalidade e assimetria das features
- **Transformações**: Recomendações específicas para features enviesadas
- **Correlações**: Mapeamento de relações entre variáveis
- **Clusters**: Visualização PCA mostra separabilidade das classes

### Outliers e Estratégia de Tratamento
- **Métodos Múltiplos**: IQR, Z-score e Modified Z-score para detecção robusta
- **Consensus Outliers**: Outliers detectados por múltiplos métodos
- **Distribuição por Classe**: Análise de onde os outliers se concentram

## 📈 Pipeline de Pré-processamento Recomendado

### 1. **Limpeza e Preparação**
```python
# Baseado nos resultados da EDA
steps = [
    'Verificar valores ausentes (já validado)',
    'Remover outliers extremos apenas do tráfego normal',
    'Manter outliers de ataque para validação do modelo'
]
```

### 2. **Transformações de Features**
```python
# Features com alta assimetria (|skew| > 2)
transformations = {
    'feature_X': 'Log(1+x)',  # Se aplicável
    'feature_Y': 'Box-Cox',   # Se aplicável
    'feature_Z': 'Yeo-Johnson'  # Mais robusto
}
```

### 3. **Normalização**
```python
# Após transformações
normalization_strategy = 'StandardScaler'  # Recomendado para autoencoders
# Alternativa: MinMaxScaler para features já bem distribuídas
```

### 4. **Seleção de Features**
```python
# Baseado nos testes estatísticos
selected_features = [
    # Features com alta significância estatística
    # Features com baixa correlação entre si (< 0.9)
    # Features identificadas nos testes de hipótese
]
```

## 🤖 Estratégia para Autoencoder

### **Arquitetura Recomendada**
- **Input Layer**: Número de features selecionadas
- **Encoder**: Redução progressiva (ex: input → 32 → 16 → 8)
- **Bottleneck**: 3-5 neurônios (baseado na análise PCA)
- **Decoder**: Expansão simétrica (8 → 16 → 32 → input)
- **Activation**: ReLU nas camadas ocultas, linear na saída

### **Estratégia de Treinamento**
1. **Dados de Treino**: Apenas tráfego normal (label=0)
2. **Remoção de Outliers**: Remover outliers consensus apenas do tráfego normal
3. **Validação**: Split do tráfego normal (80% treino, 20% validação)
4. **Teste**: Conjunto misto (normal + ataque) para avaliar detecção

### **Threshold para Detecção**
- **Baseline**: Usar erro de reconstrução no conjunto de validação normal
- **Threshold**: Percentil 95-99 do erro de reconstrução no tráfego normal
- **Validação**: Testar com dados de ataque conhecidos

## 🔍 Próximos Passos Detalhados

### **Etapa 5: Implementação do Pipeline**
1. **Criar classe de pré-processamento** com transformações identificadas
2. **Implementar autoencoder** com arquitetura baseada na EDA
3. **Pipeline scikit-learn** para integração das etapas
4. **Validação cruzada** estratificada para robustez

### **Etapa 6: Treinamento e Validação**
1. **Métricas de Avaliação**:
   - Precisão, Recall, F1-Score
   - ROC-AUC (objetivo: >0.90)
   - Precision-Recall AUC para dados desbalanceados
2. **Análise de Threshold**:
   - Curva ROC para otimização
   - Trade-off entre falsos positivos e falsos negativos

### **Etapa 7: Interpretabilidade**
1. **Feature Importance**: Análise do impacto de cada feature no erro
2. **Visualização**: t-SNE/UMAP do espaço latente
3. **Casos de Erro**: Análise de falsos positivos/negativos

## 📊 Métricas de Sucesso Esperadas

### **Baseline Mínimo**
- **Accuracy**: >85%
- **Precision**: >80% (minimizar falsos positivos)
- **Recall**: >90% (detectar a maioria dos ataques)
- **F1-Score**: >85%

### **Objetivo Ideal**
- **ROC-AUC**: >0.95
- **PR-AUC**: >0.90
- **False Positive Rate**: <5%
- **Detection Rate**: >95%

---

## 📋 Validação da Metodologia

### 🔬 **Qualidades do Dataset**
- **Coleta Manual**: Dados autênticos coletados em ambiente controlado
- **Ataques Reais**: Executados por especialistas para garantir realismo
- **Ambiente Controlado**: Reduz ruído e garante qualidade dos dados
- **Reprodutibilidade**: Metodologia documentada permite replicação

### ⚠️ **Limitações e Considerações**
- **Escopo**: Aplicável a ambientes similares ao de coleta
- **Temporal**: Padrões de ataque evoluem - necessário retreinamento periódico
- **Generalização**: Pode necessitar ajustes para outros tipos de rede

### 🎯 **Contribuições Esperadas**
- **Metodologia**: Pipeline completo de detecção com autoencoders
- **Benchmark**: Baseline para futuros trabalhos na área
- **Reprodutibilidade**: Código e dados para validação por terceiros

## 🚀 **Comando para Próxima Etapa**

```bash
# Executar após conclusão da EDA
python scripts/train_autoencoder.py --config config/model_config.yaml
```

**Status**: ✅ EDA Completa - Pronto para Modelagem

In [ ]:
# 11. EXPORTAR TABELAS E ARTEFATOS PARA REPORTS/

print("💾 Exportando tabelas-chave para reports/...")

# Criar diretório reports se não existir
import os
os.makedirs('../reports', exist_ok=True)
os.makedirs('../figures', exist_ok=True)

# 1. Exportar resultados dos testes de hipótese
if 'hypothesis_df' in locals():
    hypothesis_df.to_csv('../reports/statistical_tests_results.csv', index=False)
    print("✅ Resultados dos testes estatísticos salvos em reports/statistical_tests_results.csv")

# 2. Exportar resultados das transformações
if 'transformation_df' in locals():
    transformation_df.to_csv('../reports/feature_transformations.csv', index=False)
    print("✅ Recomendações de transformações salvas em reports/feature_transformations.csv")

# 3. Exportar dados de VIF (multicolinearidade)
if 'vif_data' in locals():
    vif_data.to_csv('../reports/vif_multicollinearity.csv', index=False)
    print("✅ Análise de multicolinearidade (VIF) salva em reports/vif_multicollinearity.csv")

# 4. Exportar matriz de correlação
correlation_matrix = df[numeric_cols].corr()
correlation_matrix.to_csv('../reports/correlation_matrix.csv')
print("✅ Matriz de correlação salva em reports/correlation_matrix.csv")

# 5. Exportar comparações estatísticas entre classes
if 'comparison_df' in locals():
    comparison_df.to_csv('../reports/class_comparison_stats.csv', index=False)
    print("✅ Comparações estatísticas entre classes salvas em reports/class_comparison_stats.csv")

# 6. Exportar análise de outliers
if 'outlier_df' in locals():
    outlier_df.to_csv('../reports/outlier_analysis.csv', index=False)
    print("✅ Análise de outliers salva em reports/outlier_analysis.csv")

# 7. Exportar contribuições das componentes principais
if 'feature_contributions' in locals():
    feature_contributions.to_csv('../reports/pca_feature_contributions.csv')
    print("✅ Contribuições PCA salvas em reports/pca_feature_contributions.csv")

# 8. Exportar resumo geral do dataset
dataset_summary = {
    'Total_Samples': len(df),
    'Total_Features': len(numeric_cols),
    'Normal_Traffic': len(df[df['label'] == 0]),
    'Attack_Traffic': len(df[df['label'] == 1]),
    'Missing_Values': df.isnull().sum().sum(),
    'Memory_Usage_MB': df.memory_usage(deep=True).sum() / 1024**2
}

summary_df = pd.DataFrame([dataset_summary])
summary_df.to_csv('../reports/dataset_summary.csv', index=False)
print("✅ Resumo do dataset salvo em reports/dataset_summary.csv")

# 9. Exportar features selecionadas para o modelo
selected_features_info = {
    'All_Features': numeric_cols,
    'Significant_Features': significant_features if 'significant_features' in locals() else [],
    'Features_Count': len(numeric_cols),
    'Significant_Count': len(significant_features) if 'significant_features' in locals() else 0
}

# Salvar como JSON para facilitar leitura pelo pipeline
import json
with open('../reports/selected_features.json', 'w') as f:
    json.dump(selected_features_info, f, indent=2)
print("✅ Features selecionadas salvas em reports/selected_features.json")

print(f"\n📊 Total de arquivos exportados: 9")
print("📁 Todos os artefatos estão disponíveis em reports/ para referência no artigo")

In [ ]:
# 12. SALVAR VISUALIZAÇÕES PRINCIPAIS EM FIGURES/

print("🎨 Salvando visualizações principais em figures/...")

# Configurações para alta qualidade
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

# 1. Salvar distribuições transformadas (recriação otimizada)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    if i < len(axes):
        # Distribuição original
        axes[i].hist(df[col], bins=50, alpha=0.7, density=True, color='skyblue', edgecolor='black')
        axes[i].set_title(f'{col} - Distribuição Original\nSkew: {df[col].skew():.2f}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Densidade')
        axes[i].grid(True, alpha=0.3)

# Remove subplot extra se houver
if len(numeric_cols) < len(axes):
    for i in range(len(numeric_cols), len(axes)):
        fig.delaxes(axes[i])

plt.suptitle('Distribuições das Features Numéricas', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/01_feature_distributions.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Distribuições das features salvas em figures/01_feature_distributions.png")

# 2. Boxplots comparativos por classe
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, col in enumerate(numeric_cols):
    if i < len(axes):
        box_data = [df[df['label'] == 0][col].dropna(), df[df['label'] == 1][col].dropna()]
        box = axes[i].boxplot(box_data, labels=['Normal', 'Ataque'], patch_artist=True)
        
        # Colorir as caixas
        colors = ['lightblue', 'lightcoral']
        for patch, color in zip(box['boxes'], colors):
            patch.set_facecolor(color)
        
        axes[i].set_title(f'{col} por Classe')
        axes[i].set_ylabel(col)
        axes[i].grid(True, alpha=0.3)

# Remove subplot extra se houver
if len(numeric_cols) < len(axes):
    for i in range(len(numeric_cols), len(axes)):
        fig.delaxes(axes[i])

plt.suptitle('Comparação de Features entre Classes Normal e Ataque', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/02_class_comparison_boxplots.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Boxplots comparativos salvos em figures/02_class_comparison_boxplots.png")

# 3. Matriz de correlação crítica
critical_threshold = 0.5
corr_matrix = df[numeric_cols].corr()
critical_corr = corr_matrix.copy()
critical_corr[abs(critical_corr) < critical_threshold] = 0

plt.figure(figsize=(12, 10))
mask = critical_corr == 0
sns.heatmap(critical_corr, annot=True, fmt='.3f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, mask=mask, 
            cbar_kws={"shrink": .8})
plt.title(f'Matriz de Correlações Críticas (|r| ≥ {critical_threshold})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/03_correlation_matrix_critical.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Matriz de correlação crítica salva em figures/03_correlation_matrix_critical.png")

# 4. Projeção PCA recriada
if len(df) > 0:
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    
    X = df[numeric_cols].copy()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    
    explained_variance = pca.explained_variance_ratio_
    
    # Subplot com múltiplas visualizações PCA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Variância explicada
    ax1.bar(range(1, len(explained_variance)+1), explained_variance, alpha=0.7, color='skyblue')
    cumulative_variance = np.cumsum(explained_variance)
    ax1.plot(range(1, len(cumulative_variance)+1), cumulative_variance, 'ro-', color='red')
    ax1.set_xlabel('Componente Principal')
    ax1.set_ylabel('Variância Explicada')
    ax1.set_title('Variância Explicada por Componente')
    ax1.grid(True, alpha=0.3)
    ax1.legend(['Acumulada', 'Individual'])
    
    # PC1 vs PC2
    normal_mask = df['label'] == 0
    attack_mask = df['label'] == 1
    
    ax2.scatter(X_pca[normal_mask, 0], X_pca[normal_mask, 1], 
               alpha=0.6, c='skyblue', label='Normal', s=30)
    ax2.scatter(X_pca[attack_mask, 0], X_pca[attack_mask, 1], 
               alpha=0.6, c='salmon', label='Ataque', s=30)
    ax2.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
    ax2.set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
    ax2.set_title('Projeção PCA (PC1 vs PC2)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Loading plot
    loadings = pca.components_[:2].T * np.sqrt(pca.explained_variance_[:2])
    ax3.scatter(loadings[:, 0], loadings[:, 1], alpha=0.7, s=100, color='purple')
    for i, feature in enumerate(numeric_cols):
        ax3.annotate(feature, (loadings[i, 0], loadings[i, 1]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
    ax3.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
    ax3.set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
    ax3.set_title('Loading Plot (Contribuição das Features)')
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    ax3.axvline(x=0, color='k', linestyle='--', alpha=0.5)
    
    # PC1 vs PC3
    ax4.scatter(X_pca[normal_mask, 0], X_pca[normal_mask, 2], 
               alpha=0.6, c='skyblue', label='Normal', s=30)
    ax4.scatter(X_pca[attack_mask, 0], X_pca[attack_mask, 2], 
               alpha=0.6, c='salmon', label='Ataque', s=30)
    ax4.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
    ax4.set_ylabel(f'PC3 ({explained_variance[2]*100:.1f}%)')
    ax4.set_title('Projeção PCA (PC1 vs PC3)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle('Análise de Componentes Principais (PCA)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../figures/04_pca_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Análise PCA salva em figures/04_pca_analysis.png")

# 5. Balanceamento das classes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Contagem por classe
label_counts = df['label'].value_counts().sort_index()
label_props = df['label'].value_counts(normalize=True).sort_index()

balance_data = pd.DataFrame({
    'Classe': ['Normal (0)', 'Ataque (1)'],
    'Quantidade': label_counts.values,
    'Proporção (%)': (label_props.values * 100).round(2)
})

# Gráfico de barras
balance_data.plot(x='Classe', y='Quantidade', kind='bar', ax=ax1, color=['skyblue', 'salmon'], legend=False)
ax1.set_title('Distribuição Absoluta das Classes', fontweight='bold')
ax1.set_xlabel('Classe')
ax1.set_ylabel('Número de Fluxos')
ax1.tick_params(axis='x', rotation=0)

# Gráfico de pizza
ax2.pie(balance_data['Quantidade'], labels=balance_data['Classe'], autopct='%1.1f%%', 
        colors=['skyblue', 'salmon'], startangle=90)
ax2.set_title('Proporção das Classes', fontweight='bold')

plt.suptitle('Análise de Balanceamento do Dataset', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/05_class_balance.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Análise de balanceamento salva em figures/05_class_balance.png")

# 6. Pairplot simplificado (principais features)
if len(numeric_cols) >= 3:
    # Selecionar 3 features mais importantes para pairplot (economizar espaço)
    top_features = numeric_cols[:3]  # Primeiras 3 features
    
    g = sns.pairplot(data=df[top_features + ['label']], 
                     hue='label', 
                     plot_kws={'alpha': 0.6, 's': 30},
                     diag_kind='hist',
                     palette=['skyblue', 'salmon'])
    
    g.fig.suptitle('Análise Bivariada - Top 3 Features', y=1.02, fontsize=14, fontweight='bold')
    
    # Personalizar legendas
    for ax in g.axes.flat:
        if ax.legend_:
            ax.legend(labels=['Normal', 'Ataque'], loc='best')
    
    plt.savefig('../figures/06_pairplot_top_features.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Pairplot das principais features salvo em figures/06_pairplot_top_features.png")

print(f"\n🎨 Total de visualizações salvas: 6")
print("📁 Todas as visualizações estão disponíveis em figures/ para inserção no manuscrito")

# Reset das configurações do matplotlib
plt.rcParams.update(plt.rcParamsDefault)